# Bayesian Optimization with a Hessian
Here we demonstrate the use of a Hessian matrix to estimate the kernel.

## Specifiying generator options
We start with the generator defaults and add a hessian kernel to the model. This also
 requires specifying that we will not normalize inputs to the GP model. Note: this
 can potentially mess up training of other hyperparameters.

In [ ]:
# set values if testing
import os
import torch
from copy import deepcopy
from xopt import Xopt, Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from xopt.generators.bayesian.custom_botorch.hessian_kernel import HessianRBF
from gpytorch.kernels import ScaleKernel
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20

vocs = deepcopy(tnk_vocs)
vocs.objectives = {"y2": "MINIMIZE"}

# define a custom kernel and create the model constructor
hessian_matrix = torch.tensor([[1, -0.8], [-0.8, 1]]).double()
kernel = ScaleKernel(HessianRBF(hessian_matrix))
gp_constructor = StandardModelConstructor(covar_modules={"y2": kernel})


generator = UpperConfidenceBoundGenerator(vocs=vocs, gp_constructor=gp_constructor)
generator.numerical_optimizer.n_restarts = NUM_RESTARTS
generator.n_monte_carlo_samples = NUM_MC_SAMPLES

evaluator = Evaluator(function=evaluate_TNK)

X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X

## Evaluate explict points and view model
We start with evaluating 2 points that we know satisfy the constraints. Note the
cross correlations between x1 and x2 due to the Hessian kernel.

In [ ]:
X.evaluate_data({"x1": [1.0, 0.75], "x2": [1.0, 2.0]})
X.generator.train_model()
fig, ax = X.generator.visualize_model(show_feasibility=True, n_grid=100)

In [ ]:
X.data